# Math 134C Week 5

In [1]:
import math
import numpy as np
from scipy.stats import norm

def NPV(r, cf, continuous_compounding = False, dt = None): 
    if dt is None: dt = range(len(cf))
    pv = 0
    if not continuous_compounding:
        for k in range(0, len(cf)):
            pv += cf[k] / (1+r)**dt[k]
    else:
        for k in range(0, len(cf)):
            pv += cf[k] * math.exp(-r * dt[k])
    return pv

# r is the risk-free rate, F is a cash flow, and dT is the time that we get each cash flow
def PV(F, dT, r):
    pv = 0
    if len(dT) != len(F):
        print("Input Error")
        return 0
    for n in range(0, len(F)):
        pv += F[n] * math.exp(-r * dT[n])
    return pv
        
# C_t is the price of call option, S_t is the current stock price, K is the strike, r is the risk-free rate, dt is time to expiration. D is the dividend of the stock, and dT is the time at which we receive the dividend. The delta is for the continuous dividend paying model. The function will return the price of the corresponding put option. 
def put_given_call(C_t, S_t, K, r, dt, D= None, dT = None, delta = 0):
    if D is None: D = []
    if dT is None: dT = []
    return C_t - (S_t - PV(D, dT, r)) * math.exp(-delta * dt) + K * math.exp(-r * dt)

def call_given_put(P_t, S_t, K, r, dt, D = None, dT = None, delta = 0):
    if D is None: D = []
    if dT is None: dT = []
    return P_t + (S_t - PV(D, dT, r)) * math.exp(-delta * dt) - K * math.exp(-r * dt)

def one_step_binomial(r, dt, u, d, Vu, Vd, delta = 0):
    q = (math.exp((r - delta) * dt) - d)/(u-d)
    if not (1 > q > 0): 
        print("Arbitrage opportunity exists")
        return -1
    return math.exp(-r * dt)*(q*Vu+(1-q)*Vd)    

def Delta(r, dt, u, d, Vu, Vd, S, delta = 0):
    return math.exp(-delta * dt)*(Vu-Vd)/(S*(u-d))

def B(r, dt, u, d, Vu, Vd):
    return math.exp(-r * dt)*(u * Vd - d * Vu)/(u-d)

def black_scholes_call(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))
    d2 = d1 - sigma * np.sqrt(dt)
    call_price = S_t * np.exp(-delta * dt) * norm.cdf(d1) - K * np.exp(-r * dt) * norm.cdf(d2)
    return call_price

def black_scholes_put(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))
    d2 = d1 - sigma * np.sqrt(dt)
    call_price = - S_t * np.exp(-delta * dt) * norm.cdf(-d1) + K * np.exp(-r * dt) * norm.cdf(-d2)
    return call_price


# Black-Scholes Formula
Assume that the risk-free rate is $r$. Then the call option with strike $K$, mature in time T, on a stock with price $S$, volatility $\sigma$, continuous dividend yield $\delta$ has price 
$$C = Se^{-\delta T}N(d_1)-Ke^{-rT}N(d_2)$$
$$d_1 = \frac{\ln(S/K)+(r-\delta+\frac{1}{2}\sigma^2)T}{\sigma\sqrt{T}}$$
$$d_2 = d_1-\sigma\sqrt{T}$$

The corresponding put option has price
$$P = Ke^{-rT}N(-d_2)-Se^{-\delta T}N(-d_1)$$

**Quiz 4.1**
Assume the Black-Scholes framework for option pricing.

For a nond ividend-paying stock and a European call option on the stock, you are given:

- The volatility of the stock is 20%
- The continuously compounded annual risk-free interest rate is 4%
- The time to maturity of the option is six months
- The Black-Scholes price of the call option is $9.94
- d1 = 0.21213

Calculate the strike price of the call option.

**Solution:** \
We first solve $S/K$ from the equation $d_1 = \frac{\ln(S/K)+(r-\delta+\frac{1}{2}\sigma^2)T}{\sigma\sqrt{T}}$. Then we rewrite $S$ as $\frac{S}{K}\cdot K$ in the formula for the call option price, and solve $K$ from the equation. 

In [12]:
d1 = 0.21213
C = 9.94
sigma = 0.2
T = 0.5
r = 0.04

d2 = d1 - sigma * np.sqrt(T)
S_over_K = math.exp(sigma * np.sqrt(T) * d1 - (r + 0.5 * sigma ** 2)*T)

K = C/(S_over_K * norm.cdf(d1) - math.exp(-r * T) * norm.cdf(d2))
print(f"The strike of this call option is {K:.2f}")

The strike of this call option is 149.99


**Quiz 4.2**
Assume the Black-Scholes framework for option pricing.

For a two-year European call option on a non dividend-paying stock, you are given:

- The current price of the stock is 98.
- The strike price of the call option is 100.
- The stock’s volatility is 32%.
- The continuously compounded annual risk-free interest rate is 5%.

Calculate the price of the call option.

In [14]:
print(f"The call option has price {black_scholes_call(98, 100, 2, 0.05, 0.32, 0):.2f}")

The call option has price 20.89


**Quiz 4.3**
Assume the Black-Scholes framework for options pricing.

Let S(t) denote the price at time t of a stock that pays no dividends. Consider a European put option with expiration date T and exercise price $K = S(0)e^{rT}$ where $r$ is the continuously compounded annual risk-free interest rate.

You are given:
- $T = 4$
- $\sigma^2 = 0.25$

Let $P(0)$ denote the price of the put option at time 0. Calculate the ratio $P(0)/S(0)$



**Solution:**
We have 
$$P = Ke^{-rT}N(-d_2)-Se^{-\delta T}N(-d_1) = \frac{K}{S}\cdot S e^{-rT}N(-d_2)-Se^{-\delta T}N(-d_1)$$
By substituting $K/S = e^{rT}$ and $\delta = 0$, we get 
$$P = SN(-d_2)-SN(-d_1)$$
and thus
$$P/S = N(-d_2)-N(-d_1)$$

Further, we have 
$$d_1 = \frac{\ln(S/K)+(r+0.5\cdot\sigma^2)T}{\sigma \sqrt{T}} = \frac{-rT+(r+0.5\cdot\sigma^2)T}{\sigma\sqrt{T}} = \frac{0.5\sigma^2 T}{\sigma\sqrt{T}} = 0.5\sigma\sqrt{T}$$

In [6]:
sigma = 0.5
T = 4
d1 = 0.5 * sigma * np.sqrt(T) 
d2 = d1 - sigma * np.sqrt(T)
print(f"P/S is {norm.cdf(-d2)-norm.cdf(-d1):.3f}")

P/S is 0.383


**Quiz 4.4**
Consider a one-year at-the-money European call option on a non dividend-paying stock for which the Black-Scholes framework for option pricing holds.

You are given:

- The stock currently sells for 100.
- The stock’s volatility is 30%.
- The continuously compounded risk-free interest rate is 6%.

Now consider a European put option on the same stock and subject to the same pricing parameters as the call.

You bought 1,000 units of the call and 500 units of the put. Calculate the total price of all 1,500 options, rounded to the nearest 100.

In [10]:
S = 100
sigma = 0.3
r = 0.06
T = 1
delta = 0
K = 100
print(f" The total price is {1000 * black_scholes_call(S, K, T, r, sigma, delta)+ 500 * black_scholes_put(S, K, T, r, sigma, delta):.0f}")

 The total price is 19164


**Quiz 4.5**
Assume the Black-Scholes framework for options pricing.

You are given the following information regarding two non dividend-paying stocks, Stock X and Stock Y

- The current price of Stock X and Stock Y is 50
- The volatility of Stock X is 40%
- The volatility of Stock Y is $\sigma_Y$
- A one-year at-the-money European put option on Stock X has a price of P
- A one-year at-the-money European put option on Stock Y has a price of 9.67
- The continuously compounded annual risk-free interest rate is 4%
Which of the following is correct regarding P and $\sigma_Y$?

**Solution:** \
By substituting $S = K = 50$, $r = 0.04$, $\delta = 0$, $T = 1$, we have
$$d_1 = \frac{0.04+0.5 \cdot \sigma^2}{\sigma}$$
$$d_2 = d_1 - \sigma$$

For stock $X$, we get 
$$d_1 = 0.3,\;\;d_2 = -0.1$$
Hence, we have 
$$P_X = 50\cdot e^{-0.05}N(0.1)-50\cdot N(-d_3) = 6.83$$

Since the put option on the stock $Y$ with the same stock price and strike has higher price, we know that $\sigma_Y>\sigma_X = 0.4$. 

**Quiz 4.6**
Assume the Black-Scholes framework for options pricing.

You make the following trades today to assemble a portfolio of non dividend-paying European stock options on the same underlying stock and hold all the positions until expiration.  

- Buy 1 at-the-money call option
- Sell 1 at-the-money put option
- Sell 1 call option with strike price 350
- Buy 1 put option with strike price 350

All options have an expiration date of one year from today and the continuously compounded risk-free interest rate is 2%.  The current stock price is 100 and the volatility is 30%.

Assume all premiums are exactly equal to the Black-Scholes options prices. 

What is the continuously compounded return you earn on this portfolio of options during the time that you hold it?

**Solution** \
Let $S_t$ be the payoff of our portfolio at time $t$. Then the payoff of our portfolio at expiry $T$ is 
$$(S_T-100)^+ - (100-S_T)^+ - (S_T - 350)^+ + (350 - S_T) = (S_T-100) - (S_T-350) = 250$$
The final payoff does not depend on the stock price, and by the no-arbitrage principle, the cost this portfolio is $250e^{-rT}$, and the continuously compounded return is exactly the risk-free rate $r = 0.02$. 

**Quiz 4.7**
Assume the Black-Scholes framework for options pricing.

You make the following trades today to assemble a portfolio of non dividend-paying European stock options on the same underlying stock and hold all the positions until expiration.  

- Buy 1 at-the-money call option
- Sell 1 at-the-money put option
- Sell 1 call option with strike price 350
- Buy 1 put option with strike price 350
- All options have an expiration date of one year from today and the continuously compounded risk-free interest rate is 2%.  The current stock price is 100 and the volatility is 10%.

Assume all premiums are exactly equal to the Black-Scholes options prices. 

What is the continuously compounded return you earn on this portfolio of options during the time that you hold it?

**Solution** \
Same as above

**Quiz 4.8**
You observe the price of a volatile stock over 100 trading days and record that it either doubles or halves each day with equal likelihood.

To be clear, the stock doubles (daily return of 100%) on a total of 50 days and gets cut in half (daily return of -50%) on 50 days during the 100-day observation period.

What is an accurate estimate for the stock's daily realized volatility?

The daily realized volatility is computed by 
$$\sqrt{50\cdot\ln(2)^2 + 50\cdot\ln(0.5)^2}/\sqrt{100} = 0.693$$ 

**Quiz 4.9**

Assume the Black-Scholes framework for options pricing.

You are a portfolio manager and already have a long position in Apple (ticker: AAPL).

You want to protect your long position against losses and decide to buy a European put option on AAPL with a strike price of 180.15 and an expiration date of 1-year from today.  

The continuously compounded risk-free interest rate is 8% and the stock pays no dividends. The current stock price for AAPL is 200 and its volatility is 30%.

Assume all options premiums are exactly equal to the Black-Scholes options prices. 

Once you discover how expensive the put option premium is, you decide to sell a 1-year call European call option on AAPL to help pay for the put option.  

You are free to choose the strike price of the call option.  At what strike price will the call option you sell exactly pay for the put option you buy, such that the net option premium is zero?  

Note that all option premiums should be rounded to the nearest cent ($0.01)

**Quiz 4.10**
Assume the Black-Scholes framework for options pricing.

You are a portfolio manager and already have a long position in Apple (ticker: AAPL).

The continuously compounded risk-free interest rate is 8% and the stock pays no dividends.  You also are given:

- The current stock price for AAPL is 200
- The volatility for AAPL  is 30%
- The continuously compounded expected growth rate for AAPL is $\mu= 4.5\%$

You want to protect your long position against losses and decide to hedge your long position with the following options trades:
- Buy a European put option on AAPL with an expiration date of 1-year from today with strike price of 200
- Sell a European call option on AAPL with an expiration date of 1-year from today with strike price greater than 220

The strike price of the call option is chosen carefully so that the net option premium including both options is zero. The hedged portfolio contains long 1 share of AAPL stock, long 1 put option, and short one call option.

Assume all options premiums are exactly equal to the Black-Scholes options prices. 

What is the probability that this hedged portfolio value will be strictly greater than the price of AAPL in one year?

**Solution**
The payoff of buying the put option and selling the call option is 
$$(200-S_1)^+ - (S_1-K)^+$$
where $K > 220$. The payoff is positive when $S_1<200$. Hence, the probability that the hedged portfolio value is greater than the price of the stock is 
$$P(S_1<200)$$

We know that the stock price is log-normal, and $\ln(S_1/S_0) = \ln(S_1/200)$ is a normal distribution with mean $\mu-\sigma^2/2 = 0$ and variance $\sigma^2 = 0.09$. Hence, we have 
$$P(S_1<200) = P(\ln(S_1/200)<0) = 0.5$$
since $\ln(S_1/200)$ is a normal distribution with mean $0$.

# Greeks
$\Delta = \frac{dP}{dS}$. The sensitivity to the change in stock price. We have 
$$\Delta_C = e^{-\delta T}N(d_1) > 0$$
$$\Delta_P = -e^{-\delta T}N(-d_1) < 0$$

Vega $\frac{dP}{d\sigma}$. The sensitivity to change in volatility. 
$$\frac{dC}{d\sigma} = \frac{dP}{d\sigma} = \sqrt{T}Se^{-\delta T}N‘(d_1) > 0$$

$\Gamma = \frac{d^2 P}{dS^2}$. Then second derivative of the price of the portfolio respect to change in stock price.

$\Theta = \frac{dP}{dt}$: The sensitivity to time to expiration.

$\rho = \frac{dP}{dr}$: The sensitivity to change in interest rate.